In [ ]:
#dbutils.fs.rm("/FileStore/tables", True)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,explode
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
spark = SparkSession.builder.appName("Collaborative filtering").getOrCreate()

In [ ]:
moviesDF = spark.read.options(header="True", inferSchema="True").csv("/FileStore/tables/movies.csv")
ratingsDF = spark.read.options(header="True", inferSchema="True").csv("/FileStore/tables/ratings.csv")

In [ ]:
display(moviesDF)

In [ ]:
display(ratingsDF)

In [ ]:
ratings = ratingsDF.join(moviesDF, 'movieId', 'left')

In [ ]:
(train, test) = ratings.randomSplit([0.8,0.2])

In [ ]:
ratings.count()

In [ ]:
print(train.count())
train.show()

In [ ]:
print(test.count())
test.show()

In [ ]:
als = ALS(userCol = "userId", itemCol="movieId", ratingCol="rating", nonnegative=True,implicitPrefs=False, coldStartStrategy="drop")

In [ ]:
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()

In [ ]:
evaluator = RegressionEvaluator(
           metricName="rmse", 
           labelCol="rating", 
           predictionCol="prediction")

In [ ]:
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

In [ ]:
model = cv.fit(train)
best_model = model.bestModel
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

In [ ]:
print(RMSE)

In [ ]:
recommendations = best_model.recommendForAllUsers(5)

In [ ]:
df = recommendations

In [ ]:
display(df)

In [ ]:
df2 = df.withColumn("movieid_rating", explode("recommendations"))

In [ ]:
display(df2)

In [ ]:
display(df2.select("userId", col("movieid_rating.movieId"), col("movieid_rating.rating")))